In [ ]:
#default_exp core

# Core

> Handle FHIR server communication and low-level resource functions.

In [ ]:
#export
import requests
from datetime import datetime, timezone

In [ ]:
import json

In [ ]:
#export
def get_bundle_as_raw_json(api_base, resource_type, url_suffix=None):
    "GET a bundle of resources of a specific type"
    url=f'{api_base}/{resource_type}'
    if url_suffix is not None:
        url+=url_suffix
    print('GET',url)
    return requests.get(url).json()

In [ ]:
#export
def get_next_bundle_as_raw_json(json_response):
    "GET the next set of results"
    if len(json_response['link']) < 2: return None
    url = json_response['link'][1]['url']
    print('GET',url)
    return requests.get(url).json()

In [ ]:
#export
def timestamp_now():
    return datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')

In [ ]:
#export
def new_bundle(bundle_type='collection'):
    return dict(resourceType='Bundle', 
                type=bundle_type, 
                timestamp=timestamp_now(),
                entry=[])

In [ ]:
# this will fail if the bundle we create cannot be converted to JSON
json.dumps(new_bundle())

'{"resourceType": "Bundle", "type": "collection", "timestamp": "2021-09-10T09:41:59Z", "entry": []}'

In [ ]:
# Set the base URL of the FHIR server that we will use in tests
api_base = 'http://hapi.fhir.org/baseR4'

The following cell shows how `get_bundle_as_raw_json` and `get_next_bundle_as_raw_json` can be used to read paged search results.

In [ ]:
json_response = get_bundle_as_raw_json(api_base, 'ResearchStudy')
page_count = 1
while json_response is not None:    
    for entry in json_response['entry']:
        resource = entry.get('resource', {})
        print('ResearchStudy:id', resource.get('id', 'missing'), resource.get('title', '')[:60])
    if page_count > 2: break # pull 3 pages at most to make testing fast
    json_response = get_next_bundle_as_raw_json(json_response)
    page_count += 1

GET http://hapi.fhir.org/baseR4/ResearchStudy
ResearchStudy:id 2492775 Double blind, placebo-controlled trial of a new class of art
ResearchStudy:id 1164317 Adjuvant Aspirin Treatment in PIK3CA Mutated Colon Cancer Pa
ResearchStudy:id 1164321 A Phase III, Multicenter, Randomized, Open-Label Study Compa
ResearchStudy:id 2140627 
ResearchStudy:id 2126241 Projectathon 2021 Machbarkeitsanfrage
ResearchStudy:id 2126240 Projectathon 2021 Machbarkeitsanfrage
ResearchStudy:id 2114074 Safety and Efficacy of the Xanomeline Transdermal Therapeuti
ResearchStudy:id 2114070 Safety and Efficacy of the Xanomeline Transdermal Therapeuti
ResearchStudy:id 2111327 Safety and Efficacy of the Xanomeline Transdermal Therapeuti
ResearchStudy:id 2111323 Safety and Efficacy of the Xanomeline Transdermal Therapeuti
ResearchStudy:id 2111319 Safety and Efficacy of the Xanomeline Transdermal Therapeuti
ResearchStudy:id 2111315 Safety and Efficacy of the Xanomeline Transdermal Therapeuti
ResearchStudy:id 2111311 Saf

In [ ]:
#export
def extract_references_from_resource(resource, field_name):
    "Return a list of references extracted from a single resource and field"
    result = []
    if field_name in resource:
        references = resource[field_name]
        if not isinstance(references, list): references = [references]
        for r in references:
            if 'reference' in r: 
                # TODO: check that we have a relative reference or handle other kinds too
                result.append(r['reference'])
    return result

In [ ]:
with open('test/patient_medication_bundle_0c4a1143-8d1c-42ed-b509-eac97d77c9b2.json') as f:
    test_bundle = json.load(f)
test_entry = test_bundle['entry'][3]
test_resource = test_entry['resource'] # resource with medicationReference

In [ ]:
assert (['Medication/bac1387e-3655-4e03-982f-7210faa21ea8'] 
        == extract_references_from_resource(test_resource, 'medicationReference'))

In [ ]:
#export
def extract_references(bundle, field_names):
    "Return a list of relative references e.g. `['Condition/1ddef4ad-fb76-46d6-9f1d-8ed58b173ee8']`"
    result = []
    for entry in bundle['entry']:
        resource = entry.get('resource', {})
        for f in field_names:
            result.extend(extract_references_from_resource(resource, f))
    return list(set(result)) # de-duplicate but still return a list

In [ ]:
assert (['Condition/1ddef4ad-fb76-46d6-9f1d-8ed58b173ee8', 'Medication/bac1387e-3655-4e03-982f-7210faa21ea8']
        == extract_references(test_bundle, ['medicationReference', 'reasonReference']))
# show that references are de-duplicated
test_bundle = new_bundle()
test_bundle['entry'].extend([test_entry, test_entry]) # create a bundle with duplicate medication references
assert (['Medication/bac1387e-3655-4e03-982f-7210faa21ea8']
        == extract_references(test_bundle, ['medicationReference', 'reasonReference']))

In [ ]:
#export
def get_by_reference(api_base, reference):
    "Return a resource read from a FHIR server by reference, as a list containg a single bundle entry"
    resource_type, id = reference.split('/')
    single_resource_bundle = get_bundle_as_raw_json(api_base, resource_type, f'?_id={id}')
    total = single_resource_bundle['total']
    if total != 1:
        raise Exception(f'Expected a single resource for {reference} but found {total}')
        # TODO: we might want to print a warning and return an empyt list if this is too strict
    return single_resource_bundle['entry']

If a resource has a reference like
```
'medicationReference': {'reference': 'Medication/bac1387e-3655-4e03-982f-7210faa21ea8'},
```
we can HTTP GET the referenced resource (a `Medication` in this case) with http://hapi.fhir.org/baseR4/Medication?_id=bac1387e-3655-4e03-982f-7210faa21ea8

In [ ]:
test_id = 'bac1387e-3655-4e03-982f-7210faa21ea8'
assert get_by_reference(api_base, f'Medication/{test_id}')[0]['resource']['id'] == test_id

GET http://hapi.fhir.org/baseR4/Medication?_id=bac1387e-3655-4e03-982f-7210faa21ea8
